In [3]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import time
from datetime import datetime
import xmltodict as xtd
import xml.etree.cElementTree as ET
import zillow

# API Keys
from config import gkey, zkey, zws_id

In [4]:
# target_city = input("Please input the city where you would like to move : ")
# radius=int(input("Please input the radius around the city : "))

target_city = 'atlanta'

params = {
    "address": target_city,
    "key": gkey
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# Run request
res = requests.get(base_url, params=params).json()

# Convert to JSON
# target_geo = response.json()
# pprint(target_geo)

# Extract lat/lng
lat = res["results"][0]["geometry"]["location"]["lat"]
lng = res["results"][0]["geometry"]["location"]["lng"]
state = res["results"][0]["address_components"][2]["short_name"]

z_params = {
    "latlng":f"{lat},{lng}",
    "result_type":"postal_code",
    "key":gkey
}

pcode_query = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json", params=z_params).json()
zip_code = pcode_query['results'][0]['address_components'][0]['long_name']
# Print results
print(f"city:   {target_city}\nlatlng: {lat}, {lng}\nstate:  {state}\nzip:    {zip_code}")

city:   atlanta
latlng: 33.7489954, -84.3879824
state:  GA
zip:    30334


In [5]:
params = {
    "input": "target_city",
    "inputtype":"textquery",
    "key": gkey
}

url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

In [22]:
params = {
    "address":zip_code,
    "childtype":"neighborhood",
    "zws-id":zws_id
}

url = "http://www.zillow.com/webservice/GetRegionChildren.htm"
res = requests.get(url, params=params)

In [24]:
doc = xtd.parse(res.text)
# doc["RegionChildren:regionchildren"]['response']
doc

OrderedDict([('RegionChildren:regionchildren',
              OrderedDict([('@xsi:schemaLocation',
                            'http://www.zillow.com/static/xsd/RegionChildren.xsd https://www.zillowstatic.com/vstatic/18f7df5/static/xsd/RegionChildren.xsd'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('@xmlns:RegionChildren',
                            'http://www.zillow.com/static/xsd/RegionChildren.xsd'),
                           ('request',
                            OrderedDict([('childtype', 'neighborhood')])),
                           ('message',
                            OrderedDict([('text',
                                          'Error: invalid set of parameters'),
                                         ('code', '501')]))]))])

In [21]:
df = pd.DataFrame(doc['RegionChildren:regionchildren']['response']['list']['region'])
df['url'].values

array(['http://www.zillow.com/local-info/GA-Atlanta/Midtown/r_269381/',
       'http://www.zillow.com/local-info/GA-Atlanta/Downtown/r_269306/',
       'http://www.zillow.com/local-info/GA-Atlanta/Old-Fourth-Ward/r_269397/',
       'http://www.zillow.com/local-info/GA-Atlanta/North-Buckhead/r_269391/',
       'http://www.zillow.com/local-info/GA-Atlanta/Morningside----Lenox-Park/r_274592/',
       'http://www.zillow.com/local-info/GA-Atlanta/Pine-Hills/r_403405/',
       'http://www.zillow.com/local-info/GA-Atlanta/Grant-Park/r_269330/',
       'http://www.zillow.com/local-info/GA-Atlanta/Home-Park/r_269345/',
       'http://www.zillow.com/local-info/GA-Atlanta/Virginia-Highland/r_275890/',
       'http://www.zillow.com/local-info/GA-Atlanta/Kirkwood/r_129705/',
       'http://www.zillow.com/local-info/GA-Atlanta/Atlanta-University-Center/r_403158/',
       'http://www.zillow.com/local-info/GA-Atlanta/Georgia-Tech/r_399508/',
       'http://www.zillow.com/local-info/GA-Atlanta/Collier-

In [9]:
api = zillow.ValuationApi()
zpid="2100641621"
detail_data = api.GetZEstimate(zws_id, zpid)
address = detail_data.full_address.get_dict()
address['zipcode']
# adf = pd.DataFrame(address, index=[0])
# adf

'90292'

In [10]:
zdf = detail_data.get_dict()
pd.DataFrame(zdf).head()

,zpid,similarity_score,links,full_address,zestimate,local_realestate,extended_data
amount,2100641621,None,NaN,NaN,2760217,NaN,NaN
amount_change_30days,2100641621,None,NaN,NaN,-25170,NaN,NaN
amount_currency,2100641621,None,NaN,NaN,USD,NaN,NaN
amount_last_updated,2100641621,None,NaN,NaN,10/07/2018,NaN,NaN
bathrooms,2100641621,None,NaN,NaN,NaN,NaN,None


In [11]:
address = "3400 Pacific Ave., Marina Del Rey, CA"
postal_code = "90292"

data = api.GetDeepSearchResults(zws_id, address, postal_code)

In [12]:
dsr = data.get_dict()
pd.DataFrame(dsr).head()

,zpid,similarity_score,links,full_address,zestimate,local_realestate,extended_data
amount,2100641621,None,NaN,NaN,2760217,NaN,NaN
amount_change_30days,2100641621,None,NaN,NaN,-25170,NaN,NaN
amount_currency,2100641621,None,NaN,NaN,USD,NaN,NaN
amount_last_updated,2100641621,None,NaN,NaN,10/07/2018,NaN,NaN
bathrooms,2100641621,None,NaN,NaN,NaN,NaN,3.0
